In [1]:
#include <iostream>
#include "xtensor/xarray.hpp"
#include "xtensor/xio.hpp"
#include "xtensor/xview.hpp"

In [2]:
xt::xarray<double> arr1 {
    {1, 2, 3},
    {2, 5, 7},
    {2, 5, 7}
};

In [3]:
arr1

1.,2.,3.
2.,5.,7.
2.,5.,7.


In [4]:
xt::xarray<double> arr2{
    {5, 6, 7}
};

In [5]:
xt::xarray<double> res1 = xt::view(arr1, 1) + arr2;
auto res2 = xt::view(arr1, 1) + arr2;

std::cout << res1 << std::endl;
std::cout << res2 << std::endl;

{{  7.,  11.,  14.}}
{{ 7.,  8.,  9.}}
